In [2]:
import torch.nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np

In [3]:
# import string library function 
import string 
from nltk.corpus import stopwords
# Storing the sets of punctuation in variable result 
result = string.punctuation

file1 = open('data/dialogues_text.txt', 'r')
lines = file1.readlines()
sentences = []
for line in lines:
    sentences.extend(line.split("__eou__")[:-1])

    


In [4]:

punc = string.punctuation + "’"
def clear_punctuation(str):    
    return ''.join([char if char not in punc else " " for char in str])

def clear_the(raw_words):
    return [word.lower() for word in raw_words if word.lower() not in nonsense_words and word!='']

nonsense_words = set(stopwords.words('english'))  

def generate_word_pairs(words, window):
    pairs = []
    for i in range(len(words)):
        for j in range(i-window,i+window+1):
            if j<0 or j>len(words)-1 or i==j:
                continue
            pairs.append((word_dict[words[i]],word_dict[words[j]]))
    return pairs


word_dict = {}
all_word_pairs = []
index = 0
for i in range(len(sentences)):
    punc_cleared_sentence = clear_punctuation(sentences[i])
    raw_words = punc_cleared_sentence.split(" ")
    clean_words = clear_the(raw_words)
    for word in clean_words:
        if word not in word_dict:
            word_dict[word] = index
            index +=1
    pairs = generate_word_pairs(clean_words,1)
    all_word_pairs.extend(pairs)
print(all_word_pairs[:10])



[(0, 1), (1, 0), (2, 3), (3, 2), (4, 5), (5, 4), (5, 6), (6, 5), (6, 7), (7, 6)]


In [5]:
vocabulary_size = len(word_dict)
default  = torch.zeros(vocabulary_size).float()
def get_input_layer(word_index):
    x = default.clone()
    x[word_index] = 1.0
    return x

input_array = []
output_array = []
for pair in all_word_pairs:
    input_array.append(get_input_layer(pair[0]))
    output_array.append(pair[1])


In [6]:
len

<function len(obj, /)>

In [7]:
from tqdm import tqdm
data_length = len(input_array)

W1 = Variable(torch.randn(5, vocabulary_size).float(), requires_grad=True)
W2 = Variable(torch.randn(vocabulary_size, 5).float(), requires_grad=True)
learning_rate = 0.02
    
one_chunk = data_length//100

for epoch in range(13):
    loss_sum = 0
    for i in tqdm(range(data_length), desc = 'Processing epoch: '+str(epoch)):
        x = Variable(input_array[i]).float()
        y = Variable(torch.from_numpy(np.array([output_array[i]])).long())


        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)
        log_softmax = F.log_softmax(z2, dim=0)
        loss = F.nll_loss(log_softmax.view(1,-1), y)
    #     print(loss.data.item())
        loss_sum += loss.data.item()
        loss.backward()
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        W1.grad.data.zero_()
        W2.grad.data.zero_()
    print(loss_sum/data_length)
print(W1)


Processing epoch: 0: 100%|████████████| 874114/874114 [13:33<00:00, 1074.38it/s]


8.757673978655594


Processing epoch: 1: 100%|████████████| 874114/874114 [13:41<00:00, 1064.21it/s]


7.7253003469827775


Processing epoch: 2: 100%|████████████| 874114/874114 [13:41<00:00, 1063.78it/s]


...

7.2312831770001464


Processing epoch: 11: 100%|███████████| 874114/874114 [13:40<00:00, 1065.03it/s]


7.218563580723355


Processing epoch: 12: 100%|███████████| 874114/874114 [13:40<00:00, 1065.66it/s]

7.207414930532337
tensor([[ 0.6443,  0.9156,  0.9494,  ...,  1.0956, -0.2383,  1.3978],
        [-0.2931,  0.5123,  0.0823,  ..., -0.0930, -0.0621, -1.2059],
        [-0.0457,  0.3379,  0.0277,  ...,  0.6331, -0.1091,  1.4287],
        [ 0.2997,  0.1923,  0.2591,  ..., -0.8510, -1.1853,  0.2448],
        [ 1.1033,  1.2611,  0.7962,  ...,  0.4268,  0.2590, -0.4464]],
       requires_grad=True)


In [8]:
torch.save(W1, 'w1.pt')
torch.save(W2, 'w2.pt')

In [20]:
input_array[0]

tensor([1., 0., 0.,  ..., 0., 0., 0.])

In [ ]:
input = torch.randn(3, 5, requires_grad=True)
>>> # each element in target has to have 0 <= value < C
>>> target = torch.tensor([1, 0, 4])
>>> output = F.nll_loss(F.log_softmax(input), target)
>>> output.backward()

In [44]:
input = torch.randn(3, 5, requires_grad=True)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
output = F.nll_loss(F.log_softmax(input), target)
output.backward()

/var/folders/kh/8lg1xxbx7rvb9lqfcrnyrflc0000gn/T/ipykernel_62342/3251475288.py:4: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.nll_loss(F.log_softmax(input), target)


In [68]:
# I am congcong luo

# #I
# (I, am)


# #am
# (am, I)
# (am, congcong)


# #congcong
# (congcong, luo)
# (congcong, am)

# # luo
# (luo, congcong)

# v1 = [
#     1,
#     0,
#     0,
#     0
# ] # I

# v2 = [
#     0,
#     1,
#     0,
#     0
# ] #am

# v3 = [
#     0,
#     0,
#     1,
#     0
# ] #congcong

# v3 = [
#     0,
#     0,
#     0,
#     1
# ] #luo

# input_array = [v1, v2, v2, v3, v3, v4]
# output_array = [v2, v1, v3, v2, v4, v3]






238625


dict_keys(['kitchen', 'kitchen stinks', 'stinks', 'throw', 'throw garbage', 'garbage', 'dick', 'dick getting', 'getting', 'getting coffee', 'coffee', 'coffee tonight', 'tonight', 'coffee honestly', 'honestly', 'honestly like', 'like', 'like kind', 'kind', 'kind stuff', 'stuff', 'come', 'come least', 'least', 'least try', 'try', 'try little', 'little', 'little besides', 'besides', 'besides cigarette', 'cigarette', 'wrong', 'wrong cigarette', 'cigarette thing', 'thing', 'thing go', 'go', 'go crazy', 'crazy', 'things', 'things still', 'still', 'still going', 'going', 'going badly', 'badly', 'badly houseguest', 'houseguest', 'getting worse', 'worse', 'worse eating', 'eating', 'eating house', 'house', 'house home', 'home', 'home tried', 'tried', 'tried talking', 'talking', 'talking goes', 'goes', 'goes one', 'one', 'one ear', 'ear', 'ear makes', 'makes', 'makes home', 'home fine', 'fine', 'fine really', 'really', 'really gets', 'gets', 'gets yesterday', 'yesterday', 'yesterday walked', 'wal

In [38]:
[char if char not in punc else "v" for char in "abc"]

['a', 'b', 'c']

In [40]:
import nltk

In [42]:
nltk.download('words')

[nltk_data] Downloading package words to /Users/yiwenluo/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [44]:
nltk.wordpunct_tokenize("So Dick , how about getting some coffee for tonight ? ")

['So',
 'Dick',
 ',',
 'how',
 'about',
 'getting',
 'some',
 'coffee',
 'for',
 'tonight',
 '?']

In [45]:
set(nltk.corpus.words.words())

{'commensality',
 'Chaenomeles',
 'hunchback',
 'osteogenous',
 'successorship',
 'tricipital',
 'metahydroxide',
 'fervid',
 'triumphator',
 'Tartarized',
 'hugely',
 'lacker',
 'paunchful',
 'Unionidae',
 'reforestization',
 'sulforicinoleate',
 'practicalism',
 'prase',
 'Covarecas',
 'stertorous',
 'baryton',
 'Thesmophoria',
 'scribbleable',
 'headmistressship',
 'esplees',
 'definition',
 'emergent',
 'inflexibleness',
 'overentreat',
 'retract',
 'unlocking',
 'underside',
 'chemitypy',
 'topographically',
 'untailorlike',
 'circumagitate',
 'ureal',
 'masterdom',
 'dragsman',
 'subsemitone',
 'demisecond',
 'Lophopoda',
 'neurilema',
 'merogenic',
 'nondeliberate',
 'monoscope',
 'repreparation',
 'unshackled',
 'uroschesis',
 'nontransgression',
 'unprincipledly',
 'lactarene',
 'unmorality',
 'bobjerom',
 'cosmetical',
 'ignipuncture',
 'personalistic',
 'nondisposal',
 'preinform',
 'Dicksonia',
 'autohypnotization',
 'displayable',
 'transferrer',
 'chepster',
 'insertive',

In [ ]:
>>> import nltk
>>> nltk.download('stopwords')
You can view the list of included stop words in NLTK using the code below:

import nltk
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))
print(stops)

In [46]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yiwenluo/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [48]:
stops = set(stopwords.words('english'))

In [49]:
print(stops)

{'while', 'more', 'they', 'mightn', 'those', 'haven', 'ain', 's', 'who', 'o', 'again', 'theirs', 'is', 'has', 'once', 've', "mustn't", 'above', 'some', 'can', 'do', 'my', 'your', 'his', 'will', 'yours', 'them', 'by', 'and', 'am', 'we', 'its', 'mustn', 'because', 'wouldn', "couldn't", "won't", 'of', 'their', "isn't", 'she', 'just', 'an', 'until', 'hers', 'during', 'other', 'll', 'her', 'hadn', 'itself', 'does', 'why', 'up', 'how', 'couldn', 'with', 'all', 'most', "should've", 'herself', "wasn't", "mightn't", 'have', 'what', "aren't", "shouldn't", "wouldn't", 'isn', "you're", 'off', 'very', 'down', 're', "you'd", 'own', 'themselves', 'where', 'ours', 'having', 'did', 'should', 'shan', 'further', 'm', 'ourselves', "it's", 'didn', 'here', 'y', 'these', 'won', 'weren', 'he', "that'll", 'at', 'about', 'the', 'than', 'himself', 'our', 'same', 'not', 'only', 'then', 'you', 'over', 'before', 'nor', 'be', 'it', 'so', "she's", "you'll", 'a', 'aren', 't', "weren't", 'shouldn', 'needn', 'as', 'too'